# ♞ Top Chess Players: Data Wrangling

Performed Data Wrangling for Top Chess Players Data. Cleaned Data Was Used for Further Analysis and Visualization with Tableau.

1. Removed rows with missing values (Removed = 794 rows)
2. Removed duplicate rows (removed = 35666 rows)
3. Removed players without official FIDE title (removed = 78150 rows)
4. Removed players without country (removed = 1312 rows)
5. Removed `profilelink` and `profileimage` columns
6. Changed data types for `rank`, `rating` and `lastupdated` columns
7. Added new column `rating_category`
8. Created 2 more data frames showing: `average rating / title` and `average rating / country`

*Original data = 196664 rows*  
*Cleaned data = 81254 rows*

In [103]:
# import libraries
import pandas as pd
import numpy as np

In [104]:
# read CSV file
chess_df = pd.read_csv('TopPlayers.csv', low_memory=False)

---
## 1. Data Overview

In [105]:
chess_df.head()

,name,rank,rating,country,title,profilelink,profileimage,lastupdated
0,Magnus Carlsen,1,2830,Norway,GM,https://www.chess.com/players/magnus-carlsen,https://images.chesscomfiles.com/uploads/v1/ma...,03-12-2023
1,Fabiano Caruana,2,2796,United States,GM,https://www.chess.com/players/fabiano-caruana,https://images.chesscomfiles.com/uploads/v1/ma...,03-12-2023
2,Hikaru Nakamura,3,2788,United States,GM,https://www.chess.com/players/hikaru-nakamura,https://images.chesscomfiles.com/uploads/v1/ma...,03-12-2023
3,Ding Liren,4,2780,China,GM,https://www.chess.com/players/ding-liren,https://images.chesscomfiles.com/uploads/v1/ma...,03-12-2023
4,Ian Nepomniachtchi,5,2770,Russia,GM,https://www.chess.com/players/ian-nepomniachtchi,/bundles/web/images/user-image.007dad08.svg,03-12-2023


In [106]:
chess_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196664 entries, 0 to 196663
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          196664 non-null  object
 1   rank          196664 non-null  object
 2   rating        196664 non-null  object
 3   country       196664 non-null  object
 4   title         98538 non-null   object
 5   profilelink   196664 non-null  object
 6   profileimage  196664 non-null  object
 7   lastupdated   196664 non-null  object
dtypes: object(8)
memory usage: 12.0+ MB


In [107]:
# Check for non-numerical values
non_numeric = pd.to_numeric(chess_df['rank'], errors='coerce').isna().sum()
print('non-numeric values in rank:', non_numeric)

non-numeric values in rank: 794


In [108]:
# Filter to show rows where 'rank' is non-numeric
wrong_rank = chess_df[pd.to_numeric(chess_df['rank'], errors='coerce').isna()]
wrong_rank.head()

,name,rank,rating,country,title,profilelink,profileimage,lastupdated
99,Alexander Ipatov,e,N,Türkiye,GM,https://www.chess.com/players/alexander-vladim...,/bundles/web/images/user-image.007dad08.svg,03-12-2023
118,Anton Kovalyov,e,N,Canada,GM,https://www.chess.com/players/anton-kovalyov,/bundles/web/images/user-image.007dad08.svg,03-12-2023
125,Konstantin Landa,e,N,Russia,GM,https://www.chess.com/players/konstantin-landa,https://images.chesscomfiles.com/uploads/v1/ma...,03-12-2023
297,Sergei Zhigalko,e,N,Belarus,GM,https://www.chess.com/players/sergei-zhigalko,/bundles/web/images/user-image.007dad08.svg,03-12-2023
406,Andrey Stukopin,e,N,Russia,GM,https://www.chess.com/players/andrey-stukopin,/bundles/web/images/user-image.007dad08.svg,03-12-2023


In [109]:
a = (chess_df['rank'] == 'e').sum()
b = (chess_df['rating'] == 'N').sum()

print('Value count e: ',  a)
print('Value count N: ',  b)

Value count e:  794
Value count N:  794


In [110]:
# Check for duplicates
chess_df.duplicated().sum()

35666

**Key Findings:**

- Half of the players don't have official FIDE title.
- All data types are `object`.
- 794 rows have incorrect inputs.
- 35666 rows have duplicates.
---

## 2. Data Cleaning

In [111]:
# drop rows with wrong inputs
wrong_inputs = chess_df[(chess_df['rank'] == 'e') & (chess_df['rating'] == 'N')].index
chess_df.drop(wrong_inputs, inplace=True)

In [112]:
a = (chess_df['rank'] == 'e').sum()
b = (chess_df['rating'] == 'N').sum()

print('Value count e: ',  a)
print('Value count N: ',  b)

Value count e:  0
Value count N:  0


In [113]:
# Players without assigned country
fide_country_count = chess_df[chess_df['country'] == 'FIDE'].shape[0]
fide_country_count

1312

In [114]:
# Remove rows where the 'country' column is 'FIDE'
chess_df = chess_df[chess_df['country'] != 'FIDE']

In [115]:
# Lets see duplicate rows
duplicate_rows = chess_df[chess_df.duplicated(keep=False)]
duplicate_rows

,name,rank,rating,country,title,profilelink,profileimage,lastupdated
198,Stepan Zilka,192,2600,Czech Republic,GM,https://www.chess.com/players/stepan-zilka,/bundles/web/images/user-image.007dad08.svg,03-12-2023
200,Stepan Zilka,192,2600,Czech Republic,GM,https://www.chess.com/players/stepan-zilka,/bundles/web/images/user-image.007dad08.svg,03-12-2023
449,Dr Florian Handke,444,2536,Germany,GM,https://www.chess.com/players/dr-florian-handke,/bundles/web/images/user-image.007dad08.svg,03-12-2023
451,Dr Florian Handke,444,2536,Germany,GM,https://www.chess.com/players/dr-florian-handke,/bundles/web/images/user-image.007dad08.svg,03-12-2023
474,Robert Ruck,469,2531,Hungary,GM,https://www.chess.com/players/robert-ruck,/bundles/web/images/user-image.007dad08.svg,03-12-2023
...,...,...,...,...,...,...,...,...
196643,Carsten Wollenweber,16258,2074,Germany,NaN,https://www.chess.com/players/carsten-wollenweber,/bundles/web/images/user-image.007dad08.svg,13-12-2023
196655,Carsten Wollenweber,16258,2074,Germany,NaN,https://www.chess.com/players/carsten-wollenweber,/bundles/web/images/user-image.007dad08.svg,13-12-2023
196656,Jose Ignacio Rebole Arbea,16258,2074,Spain,NaN,https://www.chess.com/players/jose-ignacio-reb...,/bundles/web/images/user-image.007dad08.svg,13-12-2023
196657,Sven Pronk,16258,2074,Netherlands,NaN,https://www.chess.com/players/sven-pronk,/bundles/web/images/user-image.007dad08.svg,13-12-2023


In [116]:
# Remove duplicates
chess_df = chess_df.drop_duplicates(keep='last')
duplicate_count = chess_df.duplicated().sum()
print(f'Duplicates count: {duplicate_count}')

Duplicates count: 0


In [117]:
# Remove players without official FIDE title
chess_df = chess_df.dropna(subset=['title'])

In [118]:
# Remove 'profileimage' and 'profilelink' columns
chess_df.drop(['profileimage', 'profilelink'], axis=1, inplace=True)

In [119]:
# Assign correct data types
chess_df['rank'] = pd.to_numeric(chess_df['rank'])
chess_df['rating'] = pd.to_numeric(chess_df['rating'])
chess_df['lastupdated'] = pd.to_datetime(chess_df['lastupdated'], format='%d-%m-%Y')
chess_df['lastupdated'] = pd.to_datetime(chess_df['lastupdated'])

In [120]:
chess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81254 entries, 0 to 196646
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         81254 non-null  object        
 1   rank         81254 non-null  int64         
 2   rating       81254 non-null  int64         
 3   country      81254 non-null  object        
 4   title        81254 non-null  object        
 5   lastupdated  81254 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 4.3+ MB


---
## 3. Preparation for Dataviz

In [121]:
# Segmenting data based on rating
def rating_category(rating):
    if rating >= 2800:
        return '2800+'
    elif 2700 <= rating < 2800:
        return '2700-2799'
    elif 2600 <= rating < 2700:
        return '2600-2699'
    elif 2500 <= rating < 2600:
        return '2500-2599'
    else:
        return 'Below 2500'

chess_df['rating_category'] = chess_df['rating'].apply(rating_category)
chess_df.head()

,name,rank,rating,country,title,lastupdated,rating_category
0,Magnus Carlsen,1,2830,Norway,GM,2023-12-03,2800+
1,Fabiano Caruana,2,2796,United States,GM,2023-12-03,2700-2799
2,Hikaru Nakamura,3,2788,United States,GM,2023-12-03,2700-2799
3,Ding Liren,4,2780,China,GM,2023-12-03,2700-2799
4,Ian Nepomniachtchi,5,2770,Russia,GM,2023-12-03,2700-2799


In [122]:
# Average rating per country with player count
avg_country = chess_df.groupby('country')['rating'].agg(['mean', 'count']).reset_index()
avg_country['mean'] = avg_country['mean'].round().astype(int)
avg_country = avg_country.rename(columns={'mean': 'average_rating', 'count': 'player_count'})
avg_country.head()

,country,average_rating,player_count
0,Afghanistan,2021,3
1,Albania,2337,55
2,Algeria,2229,158
3,Andorra,2292,62
4,Angola,2163,88


In [123]:
# Average rating per title with player count
avg_title = chess_df.groupby('title')['rating'].agg(['mean', 'count']).reset_index()
avg_title['mean'] = avg_title['mean'].round().astype(int)
avg_title = avg_title.rename(columns={'mean': 'average_rating', 'count': 'player_count'})

avg_title.head(8)

,title,average_rating,player_count
0,CM,2160,5736
1,FM,2244,35570
2,GM,2500,13239
3,IM,2346,21022
4,WCM,2058,114
5,WFM,2105,1792
6,WGM,2234,1528
7,WIM,2168,2253


In [124]:
chess_df.head(10)

,name,rank,rating,country,title,lastupdated,rating_category
0,Magnus Carlsen,1,2830,Norway,GM,2023-12-03,2800+
1,Fabiano Caruana,2,2796,United States,GM,2023-12-03,2700-2799
2,Hikaru Nakamura,3,2788,United States,GM,2023-12-03,2700-2799
3,Ding Liren,4,2780,China,GM,2023-12-03,2700-2799
4,Ian Nepomniachtchi,5,2770,Russia,GM,2023-12-03,2700-2799
5,Wesley So,6,2762,United States,GM,2023-12-03,2700-2799
6,Leinier Dominguez Perez,7,2752,United States,GM,2023-12-03,2700-2799
7,Alireza Firouzja,8,2751,France,GM,2023-12-03,2700-2799
8,Sergey Karjakin,9,2750,Russia,GM,2023-12-03,2700-2799
9,Anish Giri,10,2749,Netherlands,GM,2023-12-03,2700-2799


In [125]:
# Export data for data visualization
chess_df.to_csv('chess_df.csv', index=False)
avg_country.to_csv('avg_country.csv', index=False)
avg_title.to_csv('avg_title.csv', index=False)

In [126]:
chess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81254 entries, 0 to 196646
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   name             81254 non-null  object        
 1   rank             81254 non-null  int64         
 2   rating           81254 non-null  int64         
 3   country          81254 non-null  object        
 4   title            81254 non-null  object        
 5   lastupdated      81254 non-null  datetime64[ns]
 6   rating_category  81254 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 5.0+ MB


---
### ♟️ [Link to Tableau Visualization](https://public.tableau.com/app/profile/mantastech/viz/chess_17027630680570/Dashboard1) ♟
---